In [1]:
import torch

# Model
m_name = 'yolov5s'
m = torch.hub.load('ultralytics/yolov5', m_name, pretrained=True)
m.eval()
# Images
data = ['zidane.jpg']  # batch of images

# Inference
results = m(data)

print(results.pred)
# Results
#results.print()
#results.show()

#results.xyxy[0]  # img1 predictions (tensor)
#results.pandas().xyxy[0] 

Using cache found in /home/ubuntu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-23 Python-3.8.10 torch-2.0.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


[tensor([[7.43290e+02, 4.83436e+01, 1.14176e+03, 7.20000e+02, 8.79860e-01, 0.00000e+00],
        [4.41990e+02, 4.37337e+02, 4.96585e+02, 7.10036e+02, 6.75119e-01, 2.70000e+01],
        [1.23051e+02, 1.93238e+02, 7.14691e+02, 7.19771e+02, 6.66692e-01, 0.00000e+00],
        [9.78990e+02, 3.13579e+02, 1.02530e+03, 4.15526e+02, 2.61517e-01, 2.70000e+01]])]


In [2]:
import time
import numpy as np
from tqdm import tqdm

import torch._dynamo
torch._dynamo.reset()
torch._dynamo.config.verbose=True

def evaluate(mod, inp):
    return mod(inp).pred

print("Compiling...")
#mode = None # 58, 55
#mode = "default" # 56, 55
#mode = "max-autotune" # 57, 55
mode = "reduce-overhead" # 58, 55
evaluate_opt = torch.compile(evaluate, mode=mode)
res = evaluate_opt(m, data)
print("Compilation Done")

for fun, desc in [(evaluate, m_name), (evaluate_opt, f"{m_name}_inductor_reduce-overhead")]:
    N = 10; i = 0
    while i < N:
        res = fun(m, data)
        i += 1

    N = 50; i = 0
    TT = []
    while i < N:
        t0 = time.time()
        res = fun(m, data)
        dur = (time.time() - t0) * 1000
        TT.append(dur)
        i += 1

    print(f"{desc},{np.mean(TT):.3f},{np.percentile(TT, 50):.3f}")

Compiling...


BackendCompilerFailed: debug_wrapper raised OSError: /lib/aarch64-linux-gnu/libgomp.so.1: cannot allocate memory in static TLS block


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True
